In [1]:
#Necesary imports.
import bitso
import pandas as pd
import numpy as np
from datetime import timezone
import datetime
from dateutil.tz import tzutc
import time
from fbprophet import Prophet
import matplotlib.pyplot as plt
from scipy.stats import boxcox
import warnings
warnings.filterwarnings('ignore')
import plotly.offline
from plotly import graph_objs as go

In [2]:
#Opening my api key and secret.
with open('../key.txt') as f:
    a = f.readlines()

with open('../secret.txt') as p:
    b = p.readlines()

#Using the api secret and key to call the api.
API_SECRET = [i.split('\n') for i in a][0][0]
API_KEY = [i.split('\n') for i in b][0][0]
api = bitso.Api(API_KEY, API_SECRET)

In [3]:
'''
marker for last_scrap on 9-1-20: 13133700, I save this marker so i can use it
the next time i'm scrapping
'''
#Defining the empyt lists of the elements we need from the api.
btc_mxn_date = []
btc_mxn_price = []
btc_mxn_volume = []
btc_mxn_hour = []
btc_mxn_side = []

#Marker than changes the time in the api calls.
mark = 13059700

#Datetime when the code is running
now = datetime.datetime.now(timezone.utc)

#While loop for scrapping the information we need
while 1:
    #limit of requests, the max we can get is 100
    limit = '100'
    #Calling the api trades, we use sort 'asc' so i't starts on the marker we want.
    trades = api.trades('btc_mxn', sort='asc', limit=limit, marker=mark)
    #Starting the counter so we don't raise the error of too many requests on the api.
    count = 0
    #printing the mark and the spaces, we need these so we know the last mark we use.
    print(mark)
    print('')
    print('')
    #if statement to break the while loop.
    if trades != []:
        #starting a for loop to append the elements we need from each trade to our lists
        for e in trades:
            #adding to the counter.
            count +=1
            # if statement so we just scrap elements before today.
            if e.created_at < now :
                #printing each trade so we can see that everythings is working fine.
                print(e)
                #scrapping the elements we need
                btc_mxn_price.append(e.price)
                btc_mxn_volume.append(e.amount)
                btc_mxn_hour.append(e.created_at.hour)
                btc_mxn_date.append(e.created_at.date())
                btc_mxn_side.append(e.maker_side)
        '''
        if statement that starts a timer when the count reachs 100, we can just get
        300 requests per minute so our timer waits 21 seconds before making the next
        100 requests. It also adds a thousand to our mark so it moves forward in time.
        '''
        if count == 100:
            time.sleep(21)
            count = 0
            mark = mark + 1000
            
    #else statement to break the loop.
    else:
        break
        

13059700


Trade(tid=13059703, price=137185.86, amount=0.00072894, maker_side=sell, created_at=2020-01-01 23:54:46+00:00)
Trade(tid=13059704, price=137185.71, amount=0.00036447, maker_side=sell, created_at=2020-01-01 23:54:50+00:00)
Trade(tid=13059706, price=136500.00, amount=0.03215568, maker_side=buy, created_at=2020-01-01 23:55:03+00:00)
Trade(tid=13059707, price=137185.12, amount=0.00036447, maker_side=sell, created_at=2020-01-01 23:55:04+00:00)
Trade(tid=13059714, price=137184.27, amount=0.04373679, maker_side=sell, created_at=2020-01-01 23:55:28+00:00)
Trade(tid=13059720, price=136500.00, amount=0.00737394, maker_side=buy, created_at=2020-01-01 23:56:04+00:00)
Trade(tid=13059721, price=137182.25, amount=0.00075039, maker_side=sell, created_at=2020-01-01 23:56:20+00:00)
Trade(tid=13059722, price=137181.33, amount=0.02186887, maker_side=sell, created_at=2020-01-01 23:56:46+00:00)
Trade(tid=13059727, price=137180.52, amount=0.02831306, maker_side=sell, created_at=2020-01-01 23:57:09

13060700


Trade(tid=13060701, price=134667.64, amount=0.00063427, maker_side=buy, created_at=2020-01-02 03:56:38+00:00)
Trade(tid=13060702, price=134667.65, amount=0.00063412, maker_side=sell, created_at=2020-01-02 03:56:38+00:00)
Trade(tid=13060703, price=134667.65, amount=0.00043065, maker_side=buy, created_at=2020-01-02 03:56:38+00:00)
Trade(tid=13060704, price=134667.64, amount=0.00031713, maker_side=buy, created_at=2020-01-02 03:56:41+00:00)
Trade(tid=13060705, price=134667.67, amount=0.00021532, maker_side=buy, created_at=2020-01-02 03:56:44+00:00)
Trade(tid=13060706, price=135266.83, amount=0.00147856, maker_side=sell, created_at=2020-01-02 03:56:48+00:00)
Trade(tid=13060707, price=134667.64, amount=0.00055318, maker_side=buy, created_at=2020-01-02 03:57:24+00:00)
Trade(tid=13060708, price=134667.64, amount=0.00027659, maker_side=buy, created_at=2020-01-02 03:57:26+00:00)
Trade(tid=13060709, price=134667.65, amount=0.00028487, maker_side=buy, created_at=2020-01-02 03:57:29+00:0

13061700


Trade(tid=13061749, price=135561.96, amount=0.00047860, maker_side=sell, created_at=2020-01-02 12:42:45+00:00)
Trade(tid=13061750, price=135561.46, amount=0.00028799, maker_side=sell, created_at=2020-01-02 12:43:45+00:00)
Trade(tid=13061754, price=134504.00, amount=0.07271755, maker_side=buy, created_at=2020-01-02 12:49:27+00:00)
Trade(tid=13061761, price=134507.88, amount=0.03000001, maker_side=buy, created_at=2020-01-02 12:52:56+00:00)
Trade(tid=13061763, price=135506.67, amount=0.01215416, maker_side=sell, created_at=2020-01-02 12:53:32+00:00)
Trade(tid=13061770, price=134513.62, amount=0.38657795, maker_side=buy, created_at=2020-01-02 12:57:57+00:00)
Trade(tid=13061771, price=134513.61, amount=0.65813239, maker_side=buy, created_at=2020-01-02 12:57:57+00:00)
Trade(tid=13061772, price=134515.10, amount=0.01048175, maker_side=sell, created_at=2020-01-02 12:59:19+00:00)
Trade(tid=13061773, price=135505.04, amount=0.00052261, maker_side=sell, created_at=2020-01-02 12:59:20+0

13062700


Trade(tid=13062705, price=134011.76, amount=0.00132463, maker_side=buy, created_at=2020-01-02 16:47:04+00:00)
Trade(tid=13062731, price=134388.97, amount=0.00930136, maker_side=sell, created_at=2020-01-02 16:47:15+00:00)
Trade(tid=13062807, price=134011.26, amount=0.00074621, maker_side=buy, created_at=2020-01-02 16:47:24+00:00)
Trade(tid=13062808, price=134011.00, amount=0.01492415, maker_side=buy, created_at=2020-01-02 16:47:25+00:00)
Trade(tid=13062809, price=134010.35, amount=0.00025371, maker_side=buy, created_at=2020-01-02 16:47:25+00:00)
Trade(tid=13062810, price=134010.00, amount=0.00047614, maker_side=buy, created_at=2020-01-02 16:47:25+00:00)
Trade(tid=13062814, price=134010.00, amount=0.01950400, maker_side=buy, created_at=2020-01-02 16:47:26+00:00)
Trade(tid=13062820, price=134010.00, amount=0.00001986, maker_side=buy, created_at=2020-01-02 16:47:27+00:00)
Trade(tid=13062821, price=134005.00, amount=0.00037313, maker_side=buy, created_at=2020-01-02 16:47:27+00:00

13063700


Trade(tid=13063701, price=131250.82, amount=0.03542835, maker_side=sell, created_at=2020-01-02 17:29:14+00:00)
Trade(tid=13063704, price=131250.73, amount=0.00685710, maker_side=sell, created_at=2020-01-02 17:29:40+00:00)
Trade(tid=13063733, price=131001.02, amount=0.00021213, maker_side=buy, created_at=2020-01-02 17:32:12+00:00)
Trade(tid=13063735, price=131249.99, amount=0.03809524, maker_side=sell, created_at=2020-01-02 17:32:58+00:00)
Trade(tid=13063737, price=131249.99, amount=0.00070537, maker_side=sell, created_at=2020-01-02 17:33:59+00:00)
Trade(tid=13063742, price=131200.00, amount=0.00038360, maker_side=buy, created_at=2020-01-02 17:34:35+00:00)
Trade(tid=13063743, price=131200.00, amount=0.00180000, maker_side=buy, created_at=2020-01-02 17:35:13+00:00)
Trade(tid=13063744, price=131249.99, amount=0.22857145, maker_side=sell, created_at=2020-01-02 17:35:14+00:00)
Trade(tid=13063745, price=131249.98, amount=0.01242001, maker_side=sell, created_at=2020-01-02 17:35:18+

...

13137700


Trade(tid=13137701, price=148135.00, amount=0.01811731, maker_side=buy, created_at=2020-01-09 16:24:31+00:00)
Trade(tid=13137704, price=148135.00, amount=0.07625368, maker_side=buy, created_at=2020-01-09 16:24:33+00:00)
Trade(tid=13137707, price=148135.25, amount=0.07088117, maker_side=sell, created_at=2020-01-09 16:25:05+00:00)
Trade(tid=13137710, price=148135.00, amount=0.00013740, maker_side=buy, created_at=2020-01-09 16:25:14+00:00)
Trade(tid=13137711, price=148100.00, amount=0.00006924, maker_side=buy, created_at=2020-01-09 16:25:14+00:00)
Trade(tid=13137714, price=148100.00, amount=0.01666756, maker_side=buy, created_at=2020-01-09 16:25:16+00:00)
Trade(tid=13137716, price=148135.13, amount=0.00135012, maker_side=sell, created_at=2020-01-09 16:25:25+00:00)
Trade(tid=13137721, price=148100.00, amount=0.00006723, maker_side=buy, created_at=2020-01-09 16:25:42+00:00)
Trade(tid=13137722, price=147800.00, amount=0.04459006, maker_side=buy, created_at=2020-01-09 16:25:42+00:0

13138700


Trade(tid=13138714, price=147031.03, amount=0.16884715, maker_side=buy, created_at=2020-01-09 18:22:29+00:00)
Trade(tid=13138723, price=147780.82, amount=0.00067668, maker_side=sell, created_at=2020-01-09 18:24:36+00:00)
Trade(tid=13138726, price=147600.00, amount=0.01514099, maker_side=buy, created_at=2020-01-09 18:25:21+00:00)
Trade(tid=13138727, price=147101.07, amount=0.04646841, maker_side=buy, created_at=2020-01-09 18:25:21+00:00)
Trade(tid=13138731, price=147101.15, amount=0.01018435, maker_side=buy, created_at=2020-01-09 18:25:23+00:00)
Trade(tid=13138732, price=147699.88, amount=0.01692622, maker_side=sell, created_at=2020-01-09 18:25:46+00:00)
Trade(tid=13138733, price=147101.54, amount=0.02200000, maker_side=buy, created_at=2020-01-09 18:25:47+00:00)
Trade(tid=13138736, price=147699.99, amount=0.03129641, maker_side=sell, created_at=2020-01-09 18:25:59+00:00)
Trade(tid=13138737, price=147101.76, amount=0.00769650, maker_side=buy, created_at=2020-01-09 18:26:01+00:

13139700


Trade(tid=13139701, price=150999.97, amount=0.00033330, maker_side=buy, created_at=2020-01-09 20:17:02+00:00)
Trade(tid=13139702, price=151761.06, amount=0.00004349, maker_side=sell, created_at=2020-01-09 20:17:09+00:00)
Trade(tid=13139703, price=150510.06, amount=0.00312959, maker_side=buy, created_at=2020-01-09 20:17:37+00:00)
Trade(tid=13139705, price=150510.13, amount=0.21531629, maker_side=buy, created_at=2020-01-09 20:17:46+00:00)
Trade(tid=13139707, price=150510.23, amount=0.04000709, maker_side=buy, created_at=2020-01-09 20:18:17+00:00)
Trade(tid=13139711, price=151498.23, amount=0.06600737, maker_side=sell, created_at=2020-01-09 20:18:36+00:00)
Trade(tid=13139712, price=151497.38, amount=0.02970348, maker_side=sell, created_at=2020-01-09 20:19:01+00:00)
Trade(tid=13139713, price=151495.96, amount=0.00660080, maker_side=sell, created_at=2020-01-09 20:19:38+00:00)
Trade(tid=13139714, price=151495.65, amount=0.00128056, maker_side=sell, created_at=2020-01-09 20:19:47+0

13140700


Trade(tid=13140703, price=147253.64, amount=0.00723829, maker_side=buy, created_at=2020-01-09 22:05:25+00:00)
Trade(tid=13140710, price=147700.00, amount=0.00203115, maker_side=buy, created_at=2020-01-09 22:06:22+00:00)
Trade(tid=13140711, price=147255.00, amount=0.00581305, maker_side=buy, created_at=2020-01-09 22:06:22+00:00)
Trade(tid=13140712, price=147254.05, amount=0.07078688, maker_side=buy, created_at=2020-01-09 22:06:22+00:00)
Trade(tid=13140713, price=147254.04, amount=0.32376342, maker_side=buy, created_at=2020-01-09 22:06:22+00:00)
Trade(tid=13140714, price=148192.03, amount=0.00327521, maker_side=sell, created_at=2020-01-09 22:06:35+00:00)
Trade(tid=13140715, price=147000.45, amount=0.00030813, maker_side=buy, created_at=2020-01-09 22:07:04+00:00)
Trade(tid=13140717, price=148191.23, amount=0.02988524, maker_side=sell, created_at=2020-01-09 22:07:23+00:00)
Trade(tid=13140721, price=148190.85, amount=0.00944728, maker_side=sell, created_at=2020-01-09 22:07:40+00:

13141700




In [12]:
# creating a new data frame.
df = pd.DataFrame(columns=['price', 'volume', 'hour', 'date', 'side'])
# using the lists we got from the while loop to fill the dataframe.
df.price = btc_mxn_price
df.volume = btc_mxn_volume
df.hour = btc_mxn_hour
df.date = btc_mxn_date
df.side = btc_mxn_side
df.date = df.date.astype('datetime64')
#creating new columns with the month and day of each row.
df['month'] = [i.month for i in df.date]
df['day'] = [i.day for i in df.date]

In [10]:
#Creating a function that needs a data frame and a number(month) for input.
def extraction(df, m):
    #variable that get's just the row that matches the same month
    f_df = df.loc[df['month'] == m]
    #variable that stores the year
    y = datetime.datetime.today().year
    #variable that creates a list with the numbers of days of the month
    days = list(f_df['day'].unique())
    #varible that creates a empty dictionary
    results = {}
    #list with the names of the keys in the dictionary that the function returns.
    nombres = ['date', '24h_avg', '12h_avg', 'min', 'max', 'volume', 'm_side']
    
    #for loop that creates new keys in the empty dictionary with a empty list as values.
    for i in nombres:
        results[i] = []
    
    #for that loops on each day in the list of the days of the month.
    for e in days:
        #variable that get's just the rows that match the day in the for loop.
        ndf = f_df.loc[f_df.day == e]
        #appending the results in each dictionary key.
        results['date'].append(datetime.date(y, m, e))
        results['24h_avg'].append(ndf.price.mean())
        results['12h_avg'].append(ndf.price.loc[ndf.hour<12].mean())
        results['max'].append(ndf.price.max())
        results['min'].append(ndf.price.min())
        results['volume'].append(ndf.volume.sum())
        weight = ndf.side.value_counts().sum()
        buy = ndf.side.value_counts()['buy']/weight
        sell = ndf.side.value_counts()['sell']/weight
        if buy>sell:
            results['m_side'].append('buy')
        else:
            results['m_side'].append('sell')
            
    #returns the dictionary with the appended elements
    return results

In [13]:
#creating a variable that stores the month we are in.
month = datetime.datetime.today().month
#creating a new data frame using the dictionary from the above function and the month
df_new = pd.DataFrame.from_dict(extraction(df, month))

In [14]:
#Reading our csv file where we store the past data
history = pd.read_csv('../csv_files/historic_data.csv')

In [15]:
btc_df = pd.concat([history, df_new], ignore_index=True)

In [16]:
btc_df.drop_duplicates(inplace=True)

In [17]:
btc_df.drop(btc_df.tail(1).index,inplace=True)

In [19]:
btc_df.to_csv('../csv_files/historic_data.csv', index=False)

In [20]:
btc_df.date = btc_df.date.astype('datetime64')
btc_df['min'] = btc_df['min'].astype('float')
btc_df['max'] = btc_df['max'].astype('float')
btc_df.volume = btc_df.volume.astype('float')
btc_df['m_side'] = btc_df['m_side'].astype('category')

In [21]:
df_dum = pd.get_dummies(btc_df)
df_dum.rename(columns={'m_side_buy':'buy', 'm_side_sell':'sell'}, inplace=True)

In [22]:
change = []
for i in range(1,len(df_dum)):
    if df_dum.sell.iloc[i] != df_dum.sell.iloc[i-1]:
        change.append(df_dum.date.iloc[i])

In [23]:
#Selecting the columns we need for fbprophet
btc_24h_avg = btc_df[['date', '24h_avg']].copy()
#Changing the columns names for fbprophet to accept it.
btc_24h_avg.rename(columns={'date': 'ds', '24h_avg': 'y'}, inplace=True)

In [24]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [25]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [26]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    df.dropna(inplace=True)
    
    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [27]:
train_df = btc_24h_avg.copy().set_index('ds')

In [28]:
train_df['y'], lambda_prophet = boxcox(train_df['y'])
train_df.reset_index(inplace=True)

In [34]:
m2 = Prophet(changepoints=change, seasonality_mode='multiplicative')
m2.fit(train_df)
future2 = m2.make_future_dataframe(periods=15)
forecast2 = m2.predict(future2)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [35]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column],
                                       lambda_prophet)

In [36]:
cmp_df2 = make_comparison_dataframe(btc_24h_avg, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, 15).items():
     print(err_name, err_value)

MAPE 4.385402837113806
MAE 6372.164351948198


In [37]:


def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Zona de compra', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="green"))
    upper_bound = create_go('Máximo esperado', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="red"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Venta sugerida', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Actual', 'y', num_values,
                       marker=dict(color="red"))
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='MXN'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename='btc_mxn.html', auto_open=False)

show_forecast(cmp_df2, 15, 500, 'BTC')

